# Reshaping, Reorganizing and Aggregation

In [1]:
# import pandas and NumPy
import pandas as pd
import numpy as np

# date and time functions
import datetime

# bring in matplotlib and draw inline
import matplotlib.pyplot as plt
%matplotlib inline

# Set some Pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 8)
# pd.set_option('precision', 3)

## Loading historical stock data from the web or from files

### From Web

In [2]:

import ipykernel
print(ipykernel.__version__)

6.29.4


In [3]:
# for the DataReader
# import pandas.io.data as web
import pandas_datareader.data as web

# start end end dates
start = datetime.datetime(2012, 1, 1)
end = datetime.datetime(2012, 12, 30)

# load the data
msft = web.DataReader("MSFT", 'yahoo', start, end)
aapl = web.DataReader("AAPL", 'yahoo', start, end)

# these save the data to file - optional for the examples
#msft.to_csv("msft.csv")
#aapl.to_csv("aapl.csv")

AttributeError: 'NoneType' object has no attribute 'group'

### From file

In [3]:
# read the Microsoft and Apple data from file
msft = pd.read_csv("msft.csv", index_col=0, parse_dates=True)
aapl = pd.read_csv("aapl.csv", index_col=0, parse_dates=True)

### Organizing the data for the examples

In [4]:
msft[:3]

             Open   High    Low  Close    Volume  Adj Close
Date                                                       
2012-01-03  26.55  26.96  26.39  26.77  64731500   24.42183
2012-01-04  26.82  27.47  26.78  27.40  80516100   24.99657
2012-01-05  27.38  27.73  27.29  27.68  56081400   25.25201

In [5]:
aapl[:3]

                 Open       High        Low      Close    Volume  Adj Close
Date                                                                       
2012-01-03  409.39996  412.50000  409.00000  411.22998  75555200   55.41362
2012-01-04  410.00000  414.68002  409.28000  413.44000  65005500   55.71143
2012-01-05  414.94998  418.55002  412.67004  418.03000  67817400   56.32993

# Reorganizing and reshaping data

## Concatenating data in Multiple DataFrame objects

In [13]:
# get MSFT adjusted close data for Jan and Feb 2012
msftA01 = msft['2012-01-01':'2012-01-31'][['Adj Close']]
msftA02 = msft['2012-02-01':'2012-02-29'][['Adj Close']]
msftA01[:3]

            Adj Close
Date                 
2012-01-03   24.42183
2012-01-04   24.99657
2012-01-05   25.25201

In [14]:
msftA02[:3]

            Adj Close
Date                 
2012-02-01   27.26815
2012-02-02   27.32289
2012-02-03   27.58745

In [15]:
# combine the first three rows of each of msftA01 and msftA02
pd.concat([msftA01.head(3), msftA02.head(3)])

            Adj Close
Date                 
2012-01-03   24.42183
2012-01-04   24.99657
2012-01-05   25.25201
2012-02-01   27.26815
2012-02-02   27.32289
2012-02-03   27.58745

In [16]:
# Extract only the Jan 2012 AAPL values.  
aaplA01 = aapl['2012-01-01':'2012-01-31'][['Adj Close']]
# now concat the AAPL and MSFT Jan 2012 data
# there will be duplicate index labels
withDups = pd.concat([msftA01[:3], aaplA01[:3]])
withDups

            Adj Close
Date                 
2012-01-03   24.42183
2012-01-04   24.99657
2012-01-05   25.25201
2012-01-03   55.41362
2012-01-04   55.71143
2012-01-05   56.32993

In [18]:
# show the two records for data of 2012-01-03
withDups.loc['2012-01-03']

            Adj Close
Date                 
2012-01-03   24.42183
2012-01-03   55.41362

In [19]:
# demonstrate concat with a specification of the 
# stock tickets being part of the index
# this help disambiguate the duplicate dates using
# a hierarchal index
closes = pd.concat([msftA01[:3], aaplA01[:3]], 
                    keys=['MSFT', 'AAPL'])
closes

                 Adj Close
     Date                 
MSFT 2012-01-03   24.42183
     2012-01-04   24.99657
     2012-01-05   25.25201
AAPL 2012-01-03   55.41362
     2012-01-04   55.71143
     2012-01-05   56.32993

In [24]:
# extract just MSFT values using .ix
closes.loc['MSFT'][:3]

            Adj Close
Date                 
2012-01-03   24.42183
2012-01-04   24.99657
2012-01-05   25.25201

In [29]:
closes.loc['MSFT'].loc['2012-01-04'][['Adj Close']].tolist()

[24.99657]

In [30]:
# demonstrate concatenation using two DataFrame's
# that each have two columns.  pandas will align the
# data in columns by the column names (labels)
msftAV = msft[['Adj Close', 'Volume']]
aaplAV = aapl[['Adj Close', 'Volume']]
pd.concat([msftAV, aaplAV])

            Adj Close     Volume
Date                            
2012-01-03   24.42183   64731500
2012-01-04   24.99657   80516100
2012-01-05   25.25201   56081400
2012-01-06   25.64429   99455500
...               ...        ...
2012-12-24   70.71585   43938300
2012-12-26   69.74111   75609100
2012-12-27   70.02116  113780100
2012-12-28   69.27753   88569600

[498 rows x 2 columns]

In [31]:
# demonstrate concatenation with DataFrame objects
# that do not have the same set of columns
# this demonstrates pandas filling in NaN values
aaplA = aapl[['Adj Close']]
pd.concat([msftAV, aaplA])

            Adj Close      Volume
Date                             
2012-01-03   24.42183  64731500.0
2012-01-04   24.99657  80516100.0
2012-01-05   25.25201  56081400.0
2012-01-06   25.64429  99455500.0
...               ...         ...
2012-12-24   70.71585         NaN
2012-12-26   69.74111         NaN
2012-12-27   70.02116         NaN
2012-12-28   69.27753         NaN

[498 rows x 2 columns]

In [32]:
# perform an inner join on the DataFrame's
# since aaplA does not have a Volume column, pandas
# will not include that column in the result
pd.concat([msftAV, aaplA], join='inner')

            Adj Close
Date                 
2012-01-03   24.42183
2012-01-04   24.99657
2012-01-05   25.25201
2012-01-06   25.64429
...               ...
2012-12-24   70.71585
2012-12-26   69.74111
2012-12-27   70.02116
2012-12-28   69.27753

[498 rows x 1 columns]

In [33]:
# concat along the rows, causing duplicate columns to
# be created in the result
msftA = msft[['Adj Close']]
closes = pd.concat([msftA, aaplA], axis=1)
closes[:3]

            Adj Close  Adj Close
Date                            
2012-01-03   24.42183   55.41362
2012-01-04   24.99657   55.71143
2012-01-05   25.25201   56.32993

In [34]:
# concat along rows using two DataFrame objects with
# different number of rows. This demonstrates how
# NaN values will be filled in those rows for AAPL
# which only hase three rows as compared to 5 for MSFT
pd.concat([msftAV[:5], aaplAV[:3]], axis=1,
          keys=['MSFT', 'AAPL'])

                MSFT                AAPL            
           Adj Close    Volume Adj Close      Volume
Date                                                
2012-01-03  24.42183  64731500  55.41362  75555200.0
2012-01-04  24.99657  80516100  55.71143  65005500.0
2012-01-05  25.25201  56081400  56.32993  67817400.0
2012-01-06  25.64429  99455500       NaN         NaN
2012-01-09  25.30675  59706800       NaN         NaN

In [36]:
# inner join can also be used along this axis
# this will not include rows with index labels that do
# not exist in both DataFrame objects
pd.concat([msftA[:5], aaplA[:3]], axis=1,
          join='inner', keys=['MSFT', 'AAPL'])

                MSFT      AAPL
           Adj Close Adj Close
Date                          
2012-01-03  24.42183  55.41362
2012-01-04  24.99657  55.71143
2012-01-05  25.25201  56.32993

In [37]:
# ignore indexes and just concatenate the data and
# have the result have a default integer index
pd.concat([msftA[:3], aaplA[:3]], ignore_index=True)

   Adj Close
0   24.42183
1   24.99657
2   25.25201
3   55.41362
4   55.71143
5   56.32993

## Merging DataFrame objects

In [43]:
# we will merge these two DataFrame objects, 
# so lets peek at the data to remind ourselves
# of what they contain
msftAR = msftA.reset_index()
msftVR = msft[['Volume']].reset_index()
msftAR

          Date  Adj Close
0   2012-01-03   24.42183
1   2012-01-04   24.99657
2   2012-01-05   25.25201
3   2012-01-06   25.64429
..         ...        ...
245 2012-12-24   25.38455
246 2012-12-26   25.19693
247 2012-12-27   25.29074
248 2012-12-28   24.90612

[249 rows x 2 columns]

In [44]:
msftVR

          Date    Volume
0   2012-01-03  64731500
1   2012-01-04  80516100
2   2012-01-05  56081400
3   2012-01-06  99455500
..         ...       ...
245 2012-12-24  20842400
246 2012-12-26  31631100
247 2012-12-27  39394000
248 2012-12-28  28239900

[249 rows x 2 columns]

In [39]:
msftVR[:3]

        Date    Volume
0 2012-01-03  64731500
1 2012-01-04  80516100
2 2012-01-05  56081400

In [45]:
# merge the two.  pandas finds the columns in common,
# in this case Date, and merges on that column and adds
# a column for all the other columns in both DataFrame's
msftCVR = pd.merge(msftAR, msftVR)
msftCVR[:5]

        Date  Adj Close    Volume
0 2012-01-03   24.42183  64731500
1 2012-01-04   24.99657  80516100
2 2012-01-05   25.25201  56081400
3 2012-01-06   25.64429  99455500
4 2012-01-09   25.30675  59706800

In [46]:
# we will demonstrate join semantics using this DataFrame
msftAR0_5 = msftAR[0:5]
msftAR0_5

        Date  Adj Close
0 2012-01-03   24.42183
1 2012-01-04   24.99657
2 2012-01-05   25.25201
3 2012-01-06   25.64429
4 2012-01-09   25.30675

In [47]:
# and also this one
msftVR2_4 = msftVR[2:4]
msftVR2_4

        Date    Volume
2 2012-01-05  56081400
3 2012-01-06  99455500

In [48]:
# merge semantics using default inner join
pd.merge(msftAR0_5, msftVR2_4)

        Date  Adj Close    Volume
0 2012-01-05   25.25201  56081400
1 2012-01-06   25.64429  99455500

In [49]:
# same joing but using
pd.merge(msftAR0_5, msftVR2_4, how='outer')

        Date  Adj Close      Volume
0 2012-01-03   24.42183         NaN
1 2012-01-04   24.99657         NaN
2 2012-01-05   25.25201  56081400.0
3 2012-01-06   25.64429  99455500.0
4 2012-01-09   25.30675         NaN

## Pivoting

In [69]:
msft

           Symbol   Open   High    Low  Close    Volume  Adj Close
Date                                                              
2012-01-03   MSFT  26.55  26.96  26.39  26.77  64731500   24.42183
2012-01-04   MSFT  26.82  27.47  26.78  27.40  80516100   24.99657
2012-01-05   MSFT  27.38  27.73  27.29  27.68  56081400   25.25201
2012-01-06   MSFT  27.53  28.19  27.53  28.11  99455500   25.64429
...           ...    ...    ...    ...    ...       ...        ...
2012-12-24   MSFT  27.20  27.25  27.00  27.06  20842400   25.38455
2012-12-26   MSFT  27.03  27.20  26.70  26.86  31631100   25.19693
2012-12-27   MSFT  26.89  27.09  26.57  26.96  39394000   25.29074
2012-12-28   MSFT  26.71  26.90  26.55  26.55  28239900   24.90612

[249 rows x 7 columns]

In [50]:
# need to insert Symbol column before combining
msft.insert(0, 'Symbol', 'MSFT')
aapl.insert(0, 'Symbol', 'AAPL')

# concatenate the MSFT and AAPL data
# index will consist of the Date column, which we will sort
combined = pd.concat([msft, aapl]).sort_index()

# this pushes the index into a column and resets to a 
# default integer index
s4p = combined.reset_index();
s4p[:5]

        Date Symbol       Open       High     Low      Close    Volume  \
0 2012-01-03   MSFT   26.55000   26.96000   26.39   26.77000  64731500   
1 2012-01-03   AAPL  409.39996  412.50000  409.00  411.22998  75555200   
2 2012-01-04   MSFT   26.82000   27.47000   26.78   27.40000  80516100   
3 2012-01-04   AAPL  410.00000  414.68002  409.28  413.44000  65005500   
4 2012-01-05   MSFT   27.38000   27.73000   27.29   27.68000  56081400   

   Adj Close  
0   24.42183  
1   55.41362  
2   24.99657  
3   55.71143  
4   25.25201  

In [51]:
# pivot Date into the Index, make the columns match the
# unique values in the Symbol column, and the values 
# will be the AdjClose values
closes = s4p.pivot(index='Date', columns='Symbol', 
                   values='Adj Close')
closes[:3]

Symbol          AAPL      MSFT
Date                          
2012-01-03  55.41362  24.42183
2012-01-04  55.71143  24.99657
2012-01-05  56.32993  25.25201

## Stacking and Unstacking

In [52]:
# stack the first level of columns into the index
# essentially, moves AAPL and MSFT into the index
# leaving a single colum which is the AdjClose values
stackedCloses = closes.stack()
stackedCloses

Date        Symbol
2012-01-03  AAPL      55.41362
            MSFT      24.42183
2012-01-04  AAPL      55.71143
            MSFT      24.99657
                        ...   
2012-12-27  AAPL      70.02116
            MSFT      25.29074
2012-12-28  AAPL      69.27753
            MSFT      24.90612
Length: 498, dtype: float64

In [53]:
# using .ix we can retrieve close values by
# specifying both the date and ticker
stackedCloses.loc['2012-01-03', 'AAPL']

55.41362

In [54]:
# lookup on just the date, which will give us two values
# one each for AAPL and MSFT.  
stackedCloses.loc['2012-01-03']

Symbol
AAPL    55.41362
MSFT    24.42183
dtype: float64

In [55]:
# this looks up all values for the MSFT symbol
stackedCloses.loc[:, 'MSFT']

Date
2012-01-03    24.42183
2012-01-04    24.99657
2012-01-05    25.25201
2012-01-06    25.64429
                ...   
2012-12-24    25.38455
2012-12-26    25.19693
2012-12-27    25.29074
2012-12-28    24.90612
Length: 249, dtype: float64

In [56]:
# pivots the last level of the index back into a column
unstackedCloses = stackedCloses.unstack()
unstackedCloses[:3]

Symbol          AAPL      MSFT
Date                          
2012-01-03  55.41362  24.42183
2012-01-04  55.71143  24.99657
2012-01-05  56.32993  25.25201

## Melting

In [58]:
s4p

          Date Symbol       Open       High        Low      Close     Volume  \
0   2012-01-03   MSFT   26.55000   26.96000   26.39000   26.77000   64731500   
1   2012-01-03   AAPL  409.39996  412.50000  409.00000  411.22998   75555200   
2   2012-01-04   MSFT   26.82000   27.47000   26.78000   27.40000   80516100   
3   2012-01-04   AAPL  410.00000  414.68002  409.28000  413.44000   65005500   
..         ...    ...        ...        ...        ...        ...        ...   
494 2012-12-27   AAPL  513.54004  516.25000  504.66003  515.06000  113780100   
495 2012-12-27   MSFT   26.89000   27.09000   26.57000   26.96000   39394000   
496 2012-12-28   MSFT   26.71000   26.90000   26.55000   26.55000   28239900   
497 2012-12-28   AAPL  510.28998  514.47998  508.12000  509.58997   88569600   

     Adj Close  
0     24.42183  
1     55.41362  
2     24.99657  
3     55.71143  
..         ...  
494   70.02116  
495   25.29074  
496   24.90612  
497   69.27753  

[498 rows x 8 columns]

In [57]:
# melt making id_vars of Date and Symbol, making the 
# column names the variable and the for each the value
melted = pd.melt(s4p, id_vars=['Date', 'Symbol'])
melted[:5]

        Date Symbol variable      value
0 2012-01-03   MSFT     Open   26.55000
1 2012-01-03   AAPL     Open  409.39996
2 2012-01-04   MSFT     Open   26.82000
3 2012-01-04   AAPL     Open  410.00000
4 2012-01-05   MSFT     Open   27.38000

In [59]:
# extract the values for the data for MSFT on 2012-01-03


           Date Symbol   variable         value
0    2012-01-03   MSFT       Open  2.655000e+01
498  2012-01-03   MSFT       High  2.696000e+01
996  2012-01-03   MSFT        Low  2.639000e+01
1494 2012-01-03   MSFT      Close  2.677000e+01
1992 2012-01-03   MSFT     Volume  6.473150e+07
2490 2012-01-03   MSFT  Adj Close  2.442183e+01

In [60]:
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')]

           Date Symbol   variable         value
0    2012-01-03   MSFT       Open  2.655000e+01
498  2012-01-03   MSFT       High  2.696000e+01
996  2012-01-03   MSFT        Low  2.639000e+01
1494 2012-01-03   MSFT      Close  2.677000e+01
1992 2012-01-03   MSFT     Volume  6.473150e+07
2490 2012-01-03   MSFT  Adj Close  2.442183e+01

In [65]:
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')].values

array([[Timestamp('2012-01-03 00:00:00'), 'MSFT', 'Open', 26.55],
       [Timestamp('2012-01-03 00:00:00'), 'MSFT', 'High', 26.96],
       [Timestamp('2012-01-03 00:00:00'), 'MSFT', 'Low', 26.39],
       [Timestamp('2012-01-03 00:00:00'), 'MSFT', 'Close', 26.77],
       [Timestamp('2012-01-03 00:00:00'), 'MSFT', 'Volume', 64731500.0],
       [Timestamp('2012-01-03 00:00:00'), 'MSFT', 'Adj Close', 24.42183]],
      dtype=object)

# Grouping and aggregation

## Splitting

In [67]:
combined

           Symbol       Open       High        Low      Close     Volume  \
Date                                                                       
2012-01-03   MSFT   26.55000   26.96000   26.39000   26.77000   64731500   
2012-01-03   AAPL  409.39996  412.50000  409.00000  411.22998   75555200   
2012-01-04   MSFT   26.82000   27.47000   26.78000   27.40000   80516100   
2012-01-04   AAPL  410.00000  414.68002  409.28000  413.44000   65005500   
...           ...        ...        ...        ...        ...        ...   
2012-12-27   AAPL  513.54004  516.25000  504.66003  515.06000  113780100   
2012-12-27   MSFT   26.89000   27.09000   26.57000   26.96000   39394000   
2012-12-28   MSFT   26.71000   26.90000   26.55000   26.55000   28239900   
2012-12-28   AAPL  510.28998  514.47998  508.12000  509.58997   88569600   

            Adj Close  
Date                   
2012-01-03   24.42183  
2012-01-03   55.41362  
2012-01-04   24.99657  
2012-01-04   55.71143  
...               .

In [68]:
# construct a DataFrame to demonstrate splitting
# extract from combined the Symbol and AdjClose, and reset the index
s4g = combined[['Symbol', 'Adj Close']].reset_index()
# now, add two columns, year and month, using the year and month
# portions of the data as integers
s4g.insert(1, 'Year', pd.DatetimeIndex(s4g['Date']).year)
s4g.insert(2, 'Month',pd.DatetimeIndex(s4g['Date']).month)
s4g[:5]

        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT   24.42183
1 2012-01-03  2012      1   AAPL   55.41362
2 2012-01-04  2012      1   MSFT   24.99657
3 2012-01-04  2012      1   AAPL   55.71143
4 2012-01-05  2012      1   MSFT   25.25201

In [70]:
# group by the Symbol column
s4g.groupby('Symbol')

In [71]:
# group again, but save the result this time
grouped = s4g.groupby('Symbol')
# the groupby object has a property groups, which shows how
# all rows will in mapped into the groups.   
# the type of this object is a python dict
type(grouped.groups)

pandas.io.formats.printing.PrettyDict

In [72]:
# show the mappings of rows to groups
grouped.groups

{'AAPL': [1, 3, 5, 7, 9, 11, 13, 14, 16, 18, 20, 23, 25, 27, 29, 30, 33, 34, 37, 38, 41, 43, 44, 46, 49, 51, 52, 54, 57, 59, 61, 62, 64, 67, 68, 71, 72, 75, 77, 79, 81, 82, 84, 86, 89, 90, 93, 95, 96, 99, 100, 103, 105, 107, 109, 111, 112, 114, 117, 119, 121, 122, 124, 126, 128, 130, 132, 134, 137, 139, 140, 142, 145, 147, 149, 151, 153, 154, 156, 158, 160, 163, 164, 166, 168, 170, 173, 175, 176, 178, 180, 183, 184, 186, 189, 190, 193, 194, 196, 198, ...], 'MSFT': [0, 2, 4, 6, 8, 10, 12, 15, 17, 19, 21, 22, 24, 26, 28, 31, 32, 35, 36, 39, 40, 42, 45, 47, 48, 50, 53, 55, 56, 58, 60, 63, 65, 66, 69, 70, 73, 74, 76, 78, 80, 83, 85, 87, 88, 91, 92, 94, 97, 98, 101, 102, 104, 106, 108, 110, 113, 115, 116, 118, 120, 123, 125, 127, 129, 131, 133, 135, 136, 138, 141, 143, 144, 146, 148, 150, 152, 155, 157, 159, 161, 162, 165, 167, 169, 171, 172, 174, 177, 179, 181, 182, 185, 187, 188, 191, 192, 195, 197, 199, ...]}

In [73]:
# these report the number of groups that resulted from
# the grouping
len(grouped), grouped.ngroups

(2, 2)

In [75]:
# this function will print the contents of a group
def print_groups (groupobject):
    for name, group in groupobject:
        print(name)
        print(group.head())

In [76]:
# examine our resulting groups
print_groups(grouped)

AAPL
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL   55.41362
3 2012-01-04  2012      1   AAPL   55.71143
5 2012-01-05  2012      1   AAPL   56.32993
7 2012-01-06  2012      1   AAPL   56.91880
9 2012-01-09  2012      1   AAPL   56.82851
MSFT
        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT   24.42183
2 2012-01-04  2012      1   MSFT   24.99657
4 2012-01-05  2012      1   MSFT   25.25201
6 2012-01-06  2012      1   MSFT   25.64429
8 2012-01-09  2012      1   MSFT   25.30675


In [77]:
# .size will tell us the count of items in each group
grouped.size()

Symbol
AAPL    249
MSFT    249
dtype: int64

In [78]:
# a specific group can be retrieved using .get_group()
# which returns a DataFrame representing the specified group
grouped.get_group('MSFT')

          Date  Year  Month Symbol  Adj Close
0   2012-01-03  2012      1   MSFT   24.42183
2   2012-01-04  2012      1   MSFT   24.99657
4   2012-01-05  2012      1   MSFT   25.25201
6   2012-01-06  2012      1   MSFT   25.64429
..         ...   ...    ...    ...        ...
491 2012-12-24  2012     12   MSFT   25.38455
493 2012-12-26  2012     12   MSFT   25.19693
495 2012-12-27  2012     12   MSFT   25.29074
496 2012-12-28  2012     12   MSFT   24.90612

[249 rows x 5 columns]

In [79]:
# group by three different fields and print the result
mcg = s4g.groupby(['Symbol', 'Year', 'Month'])
print_groups(mcg)

('AAPL', 2012, 1)
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL   55.41362
3 2012-01-04  2012      1   AAPL   55.71143
5 2012-01-05  2012      1   AAPL   56.32993
7 2012-01-06  2012      1   AAPL   56.91880
9 2012-01-09  2012      1   AAPL   56.82851
('AAPL', 2012, 2)
         Date  Year  Month Symbol  Adj Close
41 2012-02-01  2012      2   AAPL   61.47202
43 2012-02-02  2012      2   AAPL   61.32784
44 2012-02-03  2012      2   AAPL   61.94231
46 2012-02-06  2012      2   AAPL   62.52039
49 2012-02-07  2012      2   AAPL   63.17527
('AAPL', 2012, 3)
         Date  Year  Month Symbol  Adj Close
81 2012-03-01  2012      3   AAPL   73.36784
82 2012-03-02  2012      3   AAPL   73.46351
84 2012-03-05  2012      3   AAPL   71.84380
86 2012-03-06  2012      3   AAPL   71.45303
89 2012-03-07  2012      3   AAPL   71.51097
('AAPL', 2012, 4)
          Date  Year  Month Symbol  Adj Close
124 2012-04-02  2012      4   AAPL   83.36096
126 2012-04-03  2012      4   AA

In [80]:
# set the index of the data to be the following three fields
# we are creating a multiindex
mi = s4g.set_index(['Symbol', 'Year', 'Month'])
mi

                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03   24.42183
AAPL   2012 1     2012-01-03   55.41362
MSFT   2012 1     2012-01-04   24.99657
AAPL   2012 1     2012-01-04   55.71143
...                      ...        ...
            12    2012-12-27   70.02116
MSFT   2012 12    2012-12-27   25.29074
            12    2012-12-28   24.90612
AAPL   2012 12    2012-12-28   69.27753

[498 rows x 2 columns]

In [81]:
# now we can group based upon values in the actual index
# the following groups by level 0 of the index (Month)
mig_l1 = mi.groupby(level=0)
print_groups(mig_l1)

AAPL
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03   55.41362
            1     2012-01-04   55.71143
            1     2012-01-05   56.32993
            1     2012-01-06   56.91880
            1     2012-01-09   56.82851
MSFT
                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03   24.42183
            1     2012-01-04   24.99657
            1     2012-01-05   25.25201
            1     2012-01-06   25.64429
            1     2012-01-09   25.30675


In [82]:
# group by three levels in the index using their names
mig_l12 = mi.groupby(level=['Symbol', 'Year', 'Month'])
print_groups(mig_l12)

('AAPL', 2012, 1)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03   55.41362
            1     2012-01-04   55.71143
            1     2012-01-05   56.32993
            1     2012-01-06   56.91880
            1     2012-01-09   56.82851
('AAPL', 2012, 2)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 2     2012-02-01   61.47202
            2     2012-02-02   61.32784
            2     2012-02-03   61.94231
            2     2012-02-06   62.52039
            2     2012-02-07   63.17527
('AAPL', 2012, 3)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 3     2012-03-01   73.36784
            3     2012-03-02   73.46351
            3     2012-03-05   71.84380
            3     2012-03-06   71.45303
            3     2012-03-07   71.51097
('AAPL', 2012, 4)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2

# Aggregation

In [83]:
# this will apply the mean function to each group
mig_l12.agg(np.mean)

C:\Users\fengzhou4\AppData\Local\Temp\ipykernel_28652\1137318981.py:2: FutureWarning: The provided callable <function mean at 0x000002569398B490> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mig_l12.agg(np.mean)


                                           Date  Adj Close
Symbol Year Month                                         
AAPL   2012 1     2012-01-16 15:36:00.000000000  57.751284
            2     2012-02-14 18:00:00.000000000  67.048153
            3     2012-03-16 01:05:27.272727296  77.819703
            4     2012-04-15 16:48:00.000000000  81.659466
...                                         ...        ...
MSFT   2012 9     2012-09-16 03:47:22.105263104  28.640845
            10    2012-10-14 08:00:00.000000000  27.044293
            11    2012-11-15 18:17:08.571428608  26.004902
            12    2012-12-15 00:00:00.000000000  25.309006

[24 rows x 2 columns]

In [84]:
s4g

          Date  Year  Month Symbol  Adj Close
0   2012-01-03  2012      1   MSFT   24.42183
1   2012-01-03  2012      1   AAPL   55.41362
2   2012-01-04  2012      1   MSFT   24.99657
3   2012-01-04  2012      1   AAPL   55.71143
..         ...   ...    ...    ...        ...
494 2012-12-27  2012     12   AAPL   70.02116
495 2012-12-27  2012     12   MSFT   25.29074
496 2012-12-28  2012     12   MSFT   24.90612
497 2012-12-28  2012     12   AAPL   69.27753

[498 rows x 5 columns]

In [85]:
# example of groupby that also ignores the index
# resulting in a default integer index
# this also has the mean function applied
s4g.groupby(['Symbol', 'Year', 'Month'], 
            as_index=False).agg(np.mean)[:5]

C:\Users\fengzhou4\AppData\Local\Temp\ipykernel_28652\1393946362.py:5: FutureWarning: The provided callable <function mean at 0x000002569398B490> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  as_index=False).agg(np.mean)[:5]


  Symbol  Year  Month                          Date  Adj Close
0   AAPL  2012      1 2012-01-16 15:36:00.000000000  57.751284
1   AAPL  2012      2 2012-02-14 18:00:00.000000000  67.048153
2   AAPL  2012      3 2012-03-16 01:05:27.272727296  77.819703
3   AAPL  2012      4 2012-04-15 16:48:00.000000000  81.659466
4   AAPL  2012      5 2012-05-15 10:54:32.727272704  76.090236

In [86]:
# apply multiple functions to each group in one call
mig_l12.agg([np.mean, np.std])

C:\Users\fengzhou4\AppData\Local\Temp\ipykernel_28652\1639771455.py:2: FutureWarning: The provided callable <function mean at 0x000002569398B490> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mig_l12.agg([np.mean, np.std])
C:\Users\fengzhou4\AppData\Local\Temp\ipykernel_28652\1639771455.py:2: FutureWarning: The provided callable <function std at 0x000002569398B5B0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  mig_l12.agg([np.mean, np.std])


                                           Date                            \
                                           mean                       std   
Symbol Year Month                                                           
AAPL   2012 1     2012-01-16 15:36:00.000000000 8 days 23:07:28.349947786   
            2     2012-02-14 18:00:00.000000000 8 days 22:04:45.043185279   
            3     2012-03-16 01:05:27.272727296 9 days 01:53:36.873001463   
            4     2012-04-15 16:48:00.000000000 8 days 14:19:24.991947757   
...                                         ...                       ...   
MSFT   2012 9     2012-09-16 03:47:22.105263104 7 days 19:39:41.433791811   
            10    2012-10-14 08:00:00.000000000 8 days 19:44:08.463429077   
            11    2012-11-15 18:17:08.571428608 9 days 04:51:56.983306785   
            12    2012-12-15 00:00:00.000000000    8 days 01:29:39.069453   

                   Adj Close            
                        mean      